In [4]:
import glob, os, gc
import os.path
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from time import time
from subprocess import (call, Popen, PIPE)
from itertools import product
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
from sklearn.decomposition import (PCA, RandomizedPCA)
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from IPython.display import Image
from PIL import Image
from IPython.display import Image as IPImage
import shutil
import pybedtools
import pysam
import re
import xml.etree.ElementTree as ET
import time



##Path to Data
basepath = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/" 
diry = "final"
narrowpeak = "-ready_peaks.narrowPeak"
peaks_merged = "_peaksMerged.txt"
peaks_merged_bed = "_peaksMerged.bed"
peaks_correct_bed = "_peaksCorrect.bed"
peakregions_sml = ".peakregions_sml"
peakregions_lrg = ".peakregions_lrg"
peaks_correct_data = "_peaksCorrect.data"
loci_sml = ".loci_sml"
loci_lrg = ".loci_lrg"
overlap = "_overlap_"
overlap_sml = "_overlap_sml_"
overlap_lrg = "_overlap_lrg_"
bam = "-ready.bam"
igv = "-igv.xml"
bed = ".bed"

Bulk_Brain = "04132016_mw_Bulk_cor"
Bulk_Fibro = "05252016_mw_Bulk_fib"
SC_MDA_1 = "04132016_mw_1571_SC_B4_S48"
SC_MDA_2 = "04132016_mw_1571_SC_D4_S49"
SC_MDA_3 = "04132016_mw_L1B1_SC_A2_S43"
SC_MDA_4 = "04132016_mw_L1B1_SC_C1_S45"
SC_MDA_5 = "04132016_mw_L1B1_SC_C2_S46"
SC_MDA_6 = "04132016_mw_L1B1_SC_D2_S50"
SC_MDA_7 = "04132016_mw_L1B1_SC_E2_S51"
SC_MDA_8 = "04132016_mw_L1B1_SC_E3_S52"
SC_MDA_9 = "04132016_mw_L1B1_SC_F2_S53"
SC_MDA_10 = "04132016_mw_L1B1_SC_G1_S54"
SC_MDA_11 = "04132016_mw_L1B1_SC_H1_S55"
SC_MDA_12 = "05252016_mw_L1B1_SC_B4_S47"
SC_MALBAC_1 = "2122_S1"
SC_MALBAC_2 = "2178_S2"
SC_MALBAC_3 = "2179_S3"
SC_MALBAC_4 = "2180_S4"
SC_MALBAC_5 = "2184_S5"
SC_MALBAC_6 = "2186_S6"
SC_MALBAC_7 = "2187_S7"
SC_MALBAC_8 = "2188_S8"
SC_MALBAC_9 = "2193_S9"
SC_MALBAC_10 = "2196_S10"
SC_MALBAC_11 = "2197_S11"
SC_MALBAC_12 = "2198_S12"
SC_MALBAC_13 = "2261_S13"
SC_MALBAC_14 = "2263_S14"
SC_MALBAC_15 = "2264_S15"
SC_MALBAC_16 = "2265_S16"

## rmask Paths 
L1HS = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/rmask_L1HS_Final.bed"
L1PA2345 = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/rmask_L1PA2345_Final.bed"
L1_Other = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/rmask_L1_Other_Final.bed"
##IGV Template
IGV = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/igv-template3.xml"

def wait_timeout(proc, seconds):
    """Wait for a process to finish, or raise exception after timeout"""
    start = time.time()
    end = start + seconds
    interval = min(seconds / 1000.0, .25)

    while True:
        result = proc.poll()
        if result is not None:
            return result
        if time.time() >= end:
            proc.kill()
            #raise RuntimeError("Process timed out")
        time.sleep(interval)
        
##look at SC peaks differenatially present compared to bulk brain 
SingleCell_MALBAC = [SC_MALBAC_1,SC_MALBAC_2,SC_MALBAC_3,SC_MALBAC_4,SC_MALBAC_5,SC_MALBAC_6,SC_MALBAC_7,SC_MALBAC_8,SC_MALBAC_9,SC_MALBAC_10,SC_MALBAC_11,SC_MALBAC_12,SC_MALBAC_13,SC_MALBAC_14,SC_MALBAC_15,SC_MALBAC_16]
SingleCells = [SC_MDA_1,SC_MDA_2,SC_MDA_3,SC_MDA_4,SC_MDA_5,SC_MDA_6,SC_MDA_7,SC_MDA_8,SC_MDA_9,SC_MDA_10,SC_MDA_11,SC_MDA_12]#,SC_MALBAC_1,SC_MALBAC_2,SC_MALBAC_3,SC_MALBAC_4,SC_MALBAC_5,SC_MALBAC_6,SC_MALBAC_7,SC_MALBAC_8,SC_MALBAC_9,SC_MALBAC_10,SC_MALBAC_11,SC_MALBAC_12,SC_MALBAC_13,SC_MALBAC_14,SC_MALBAC_15,SC_MALBAC_16]

In [2]:
BulkTissue = [Bulk_Brain, Bulk_Fibro]
for tissue in BulkTissue:
    os.chdir(os.path.join(basepath, diry, tissue))
    name = os.path.join(basepath, diry, tissue, tissue)
    bamfile = os.path.join(basepath, diry, tissue, tissue+bam)
    #Call Peaks macs2

In [5]:
for cell in SingleCell_MALBAC:
    print cell
    os.chdir(os.path.join(basepath, diry, cell))
    name = os.path.join(basepath, diry, cell, cell)
    bamfile = os.path.join(basepath, diry, cell, cell+bam)
    #call peaks macs2
    p3 = Popen(['/home/preed/homer/bin/mergePeaks', '-code', '-prefix', cell, os.path.join(basepath, diry, cell, cell+narrowpeak), os.path.join(basepath, diry, Bulk_Brain, Bulk_Brain+narrowpeak), os.path.join(basepath, diry, Bulk_Fibro, Bulk_Fibro+narrowpeak)])
    p3.wait()
    myoutput = open(os.path.join(basepath, diry, cell, cell + peaks_merged), 'w')
    p4 = Popen(['/home/preed/homer/bin/mergePeaks', os.path.join(basepath, diry, cell, cell+"_100"), os.path.join(basepath, diry, cell, cell+"_110"), os.path.join(basepath, diry, cell, cell+"_101"), os.path.join(basepath, diry, cell, cell+"_010")], stdout=myoutput)
    p4.wait()
    p5 = Popen(['/home/preed/homer/bin/pos2bed.pl', os.path.join(basepath, diry, cell, cell + peaks_merged), '-o', os.path.join(basepath, diry, cell, cell+peaks_merged_bed)])
    p5.wait()
    
#Regional Encoding, nnn:nnn:n:nnn peaks found in SC,Brain,Fibro, reads found in SC, Brain, Fibro, Control or BG, overlap with L1HS,PA,or other
#add number of peaks in sml reiogn and lrg region,

2122_S1
2178_S2
2179_S3
2180_S4
2184_S5
2186_S6
2187_S7
2188_S8
2193_S9
2196_S10
2197_S11
2198_S12
2261_S13
2263_S14
2264_S15
2265_S16


In [ ]:
for cell in SingleCell_MALBAC:
    print cell
    tree = ET.parse(IGV)
    root = tree.getroot()
    root[0][0].set('path', os.path.join(basepath, diry, cell, cell + bam)) #SC Path
    root[0][2].set('path', os.path.join(basepath, diry, Bulk_Brain, Bulk_Brain + bam)) #Bulk Brain Path
    root[0][5].set('path', os.path.join(basepath, diry, Bulk_Fibro, Bulk_Fibro + bam)) #Bulk Fib Path
    root[1][0].set('id', os.path.join(basepath, diry, cell, cell + bam)) #SC Path
    root[2][0].set('id', os.path.join(basepath, diry, Bulk_Brain, Bulk_Brain + bam)) #Bulk Brain path
    root[3][0].set('id', os.path.join(basepath, diry, Bulk_Fibro, Bulk_Fibro + bam)) #Bulk Fib Path
    tree.write(os.path.join(basepath, diry, cell, cell + igv))
    
    

    myinput = open(os.path.join(basepath, diry, cell, cell + peaks_merged_bed))
    myoutput = open(os.path.join(basepath, diry, cell, cell + peaks_correct_bed), 'w')
    proc3 = Popen(['grep', '-E', '^(1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|X|Y)'], stdin=myinput, stdout=myoutput)
    proc3.wait()    
    
    myinput = os.path.join(basepath, diry, cell, cell + peaks_correct_bed)
    myoutput1 = os.path.join(basepath, diry, cell, cell + peakregions_sml)
    myoutput2 = os.path.join(basepath, diry, cell, cell + peakregions_lrg)
    with open(myoutput1, 'w') as outfile:
        with open(myinput, 'r') as infile:
            data = infile.readlines()
            for string in data:
                line = string.split('\t')
                pos1 = int(line[1])
                pos2 = int(line[2])                  
                center = int((pos1 + pos2)/2)
                pad = 2500
                start = center - pad
                end = center + pad
                row = [line[0], str(start), str(end)]
                outfile.write('\t'.join(row) + '\n')
    outfile.close()
    infile.close()   
    with open(myoutput2, 'w') as outfile:
        with open(myinput, 'r') as infile:
            data = infile.readlines()
            for string in data:
                line = string.split('\t')
                pos1 = int(line[1])
                pos2 = int(line[2])                  
                center = int((pos1 + pos2)/2)
                pad = 10000
                newstart = center - pad
                newend = center + pad
                row = [line[0], str(newstart), str(newend)]
                outfile.write('\t'.join(row) + '\n')
    outfile.close()
    infile.close()
    
    
    sc_file = pysam.AlignmentFile(os.path.join(basepath, diry, cell, cell + bam), "rb")
    bb_file = pysam.AlignmentFile(os.path.join(basepath, diry, Bulk_Brain, Bulk_Brain + bam), "rb")
    bf_file = pysam.AlignmentFile(os.path.join(basepath, diry, Bulk_Fibro, Bulk_Fibro + bam), "rb")

    myinput = os.path.join(basepath, diry, cell, cell + peaks_correct_bed)
    myoutput = os.path.join(basepath, diry, cell, cell + peaks_correct_data)
    with open(myoutput, 'w') as outfile:
        with open(myinput, 'r') as infile:
            data = infile.readlines()
            for region in data:
                sc_iter = sc_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                bb_iter = bb_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                bf_iter = bf_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                sc_i = 0
                bb_i = 0
                bf_i = 0
                for x in sc_iter: sc_i+=1
                for y in bb_iter: bb_i+=1
                for z in bf_iter: bf_i+=1
                sc_count = 1 if sc_i > 0 else 0
                bb_count = 1 if bb_i > 0 else 0 
                bf_count = 1 if bf_i > 0 else 0 
                row = [str(region.strip().split('\t')[0]), str(region.strip().split('\t')[1]), str(region.strip().split('\t')[2])]  
                outfile.write('\t'.join(row) +'\t'+str(sc_count)+str(bb_count)+str(bf_count)+'\n')
   
    filelist =[os.path.join(basepath, diry, cell, cell+narrowpeak),os.path.join(basepath, diry, Bulk_Brain, Bulk_Brain+narrowpeak),os.path.join(basepath, diry, Bulk_Fibro, Bulk_Fibro+narrowpeak),L1HS,L1PA2345,L1_Other]
    a = pybedtools.BedTool(os.path.join(basepath, diry, cell, cell + peaks_correct_bed)) ##mergedpeaks2 overlap with loci window or with peak location???
    count = 0
    for fname in filelist:
        b = pybedtools.BedTool(fname)
        a_and_b = a.intersect(b, c=True)
        myoutput = os.path.join(basepath, diry, cell, cell + overlap + str(count))
        count +=1
        a_and_b.saveas(myoutput)
        myinput = open(myoutput)
        newoutput = open(myoutput+"_binary", 'w')
        #overlap_append
        awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($7 ~ "^[1-9]*$") $7 = "1"; else $7 = $7; }; 1"""
        proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
        proc.wait()
        newoutput.flush()
    
    a_sml = pybedtools.BedTool(os.path.join(basepath, diry, cell, cell + peakregions_sml)) ##mergedpeaks2 overlap with loci window or with peak location???
    count = 0
    for fname in filelist:
        b = pybedtools.BedTool(fname)
        a_and_b = a_sml.intersect(b, c=True)
        myoutput = os.path.join(basepath, diry, cell, cell + overlap_sml + str(count))
        count +=1
        a_and_b.saveas(myoutput)
        myinput = open(myoutput)
        newoutput = open(myoutput+"_binary", 'w')
        #overlap_append
        awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
        proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
        proc.wait()
        newoutput.flush()
    
    a_lrg = pybedtools.BedTool(os.path.join(basepath, diry, cell, cell + peakregions_lrg)) ##mergedpeaks2 overlap with loci window or with peak location???
    count = 0
    for fname in filelist:
        b = pybedtools.BedTool(fname)
        a_and_b = a_lrg.intersect(b, c=True)
        myoutput = os.path.join(basepath, diry, cell, cell + overlap_lrg + str(count))
        count +=1
        a_and_b.saveas(myoutput)
        myinput = open(myoutput)
        newoutput = open(myoutput+"_binary", 'w')
        #overlap_append
        awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
        proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
        proc.wait()
        newoutput.flush()        
    
    myinput_sml = os.path.join(basepath, diry, cell, cell + peakregions_sml)
    myoutput_sml = os.path.join(basepath, diry, cell, cell + loci_sml)
    with open(myoutput_sml, 'w') as outfile:
        with open(myinput_sml, 'r') as infile:
            data = infile.readlines()
            for region in data:
                row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
                outfile.write("".join(row)+'\n')
    
    myinput_lrg = os.path.join(basepath, diry, cell, cell + peakregions_lrg)
    myoutput_lrg = os.path.join(basepath, diry, cell, cell + loci_lrg)
    with open(myoutput_lrg, 'w') as outfile:
        with open(myinput_lrg, 'r') as infile:
            data = infile.readlines()
            for region in data:
                row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
                outfile.write("".join(row)+'\n')    

    Popen(['split', '-l', '50', '-d', os.path.join(basepath, diry, cell, cell + loci_sml), os.path.join(basepath, diry, cell, cell + ".split_loci_sml_")]).wait()
    Popen(['split', '-l', '50', '-d', os.path.join(basepath, diry, cell, cell + loci_lrg), os.path.join(basepath, diry, cell, cell + ".split_loci_lrg_")]).wait()

2122_S1
2178_S2
2179_S3
2180_S4
2184_S5
2186_S6
2187_S7
2188_S8
2193_S9
2196_S10
2197_S11
2198_S12


In [4]:
for cell in SingleCell_MALBAC:
    print os.path.join(basepath, diry, cell)
    os.chdir(os.path.join(basepath, diry, cell))    
    locifile = os.path.join(basepath, diry, cell, cell + loci_sml)
    worklist = glob.glob("*.split_loci_sml_*")
    batchsize = 10
    print len(worklist)
    for i in xrange(0, len(worklist), batchsize):
        batch = worklist[i:i+batchsize]
        print i
        index = 1
        procs = []
        for file in batch:
            print file
            with open(os.path.join(basepath, diry, cell, file)) as f0:
                first = f0.readline()# Read the first line.
                for last in f0: pass
                firstpic = cell+"_sml"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                lastpic = cell+"_sml"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                if not (glob.glob(os.path.join(basepath, diry, cell, firstpic)) or glob.glob(os.path.join(basepath, diry, cell, lastpic))): 
                    p = Popen(['igv_plotter', '-o', cell+"_sml_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, diry, cell, cell + igv)])
                    procs.append(p)
        for pp in procs:
            pp.wait()
            #wait_timeout(pp,300)

/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/final/04132016_mw_1571_SC_B4_S48
48
0
04132016_mw_1571_SC_B4_S48.split_loci_sml_00
04132016_mw_1571_SC_B4_S48.split_loci_sml_01
04132016_mw_1571_SC_B4_S48.split_loci_sml_02
04132016_mw_1571_SC_B4_S48.split_loci_sml_03
04132016_mw_1571_SC_B4_S48.split_loci_sml_04
04132016_mw_1571_SC_B4_S48.split_loci_sml_05
04132016_mw_1571_SC_B4_S48.split_loci_sml_06
04132016_mw_1571_SC_B4_S48.split_loci_sml_07
04132016_mw_1571_SC_B4_S48.split_loci_sml_08
04132016_mw_1571_SC_B4_S48.split_loci_sml_09
10
04132016_mw_1571_SC_B4_S48.split_loci_sml_10
04132016_mw_1571_SC_B4_S48.split_loci_sml_11
04132016_mw_1571_SC_B4_S48.split_loci_sml_12
04132016_mw_1571_SC_B4_S48.split_loci_sml_13
04132016_mw_1571_SC_B4_S48.split_loci_sml_14
04132016_mw_1571_SC_B4_S48.split_loci_sml_15
04132016_mw_1571_SC_B4_S48.split_loci_sml_16
04132016_mw_1571_SC_B4_S48.split_loci_sml_17
04132016_mw_1571_SC_B4_S48.split_loci_sml_18
04132016_mw_1571_SC_B4_S48.split_loci_sml_19

04132016_mw_L1B1_SC_C1_S45.split_loci_sml_43
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_44
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_45
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_46
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_47
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_48
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_49
50
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_50
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_51
04132016_mw_L1B1_SC_C1_S45.split_loci_sml_52
/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/final/04132016_mw_L1B1_SC_C2_S46
39
0
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_00
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_01
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_02
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_03
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_04
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_05
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_06
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_07
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_08
04132016_mw_L1B1_SC_C2_S46.split_loci_sml_09

70
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_70
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_71
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_72
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_73
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_74
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_75
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_76
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_77
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_78
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_79
80
04132016_mw_L1B1_SC_E2_S51.split_loci_sml_80
/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/final/04132016_mw_L1B1_SC_E3_S52
66
0
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_00
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_01
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_02
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_03
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_04
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_05
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_06
04132016_mw_L1B1_SC_E3_S52.split_loci_sml_07
04132016_mw_L1B1_SC_E3_S52.split_loci_sml

04132016_mw_L1B1_SC_G1_S54.split_loci_sml_59
/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/final/04132016_mw_L1B1_SC_H1_S55
31
0
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_00
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_01
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_02
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_03
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_04
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_05
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_06
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_07
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_08
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_09
10
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_10
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_11
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_12
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_13
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_14
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_15
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_16
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_17
04132016_mw_L1B1_SC_H1_S55.split_loci_sml_18

In [2]:
for cell in SingleCell_MALBAC:
    print os.path.join(basepath, diry, cell)
    os.chdir(os.path.join(basepath, diry, cell))    
    locifile = os.path.join(basepath, diry, cell, cell + loci_lrg)
    worklist = glob.glob("*.split_loci_lrg_*")
    batchsize = 10
    print len(worklist)
    for i in xrange(0, len(worklist), batchsize):
        batch = worklist[i:i+batchsize]
        print i
        index = 1
        procs = []
        for file in batch:
            print file
            with open(os.path.join(basepath, diry, cell, file)) as f0:
                first = f0.readline()# Read the first line.
                for last in f0: pass
                firstpic = cell+"_lrg"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                lastpic = cell+"_lrg"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                if not (glob.glob(os.path.join(basepath, diry, cell, firstpic)) or glob.glob(os.path.join(basepath, diry, cell, lastpic))): 
                    p = Popen(['igv_plotter', '-o', cell+"_lrg_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, diry, cell, cell + igv)])
                    procs.append(p)
        for pp in procs:
            pp.wait()
            #wait_timeout(pp,300)

/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/final/04132016_mw_1571_SC_B4_S48
48
0
04132016_mw_1571_SC_B4_S48.split_loci_lrg_00
04132016_mw_1571_SC_B4_S48.split_loci_lrg_01
04132016_mw_1571_SC_B4_S48.split_loci_lrg_02
04132016_mw_1571_SC_B4_S48.split_loci_lrg_03
04132016_mw_1571_SC_B4_S48.split_loci_lrg_04
04132016_mw_1571_SC_B4_S48.split_loci_lrg_05
04132016_mw_1571_SC_B4_S48.split_loci_lrg_06
04132016_mw_1571_SC_B4_S48.split_loci_lrg_07
04132016_mw_1571_SC_B4_S48.split_loci_lrg_08
04132016_mw_1571_SC_B4_S48.split_loci_lrg_09
10
04132016_mw_1571_SC_B4_S48.split_loci_lrg_10
04132016_mw_1571_SC_B4_S48.split_loci_lrg_11
04132016_mw_1571_SC_B4_S48.split_loci_lrg_12
04132016_mw_1571_SC_B4_S48.split_loci_lrg_13
04132016_mw_1571_SC_B4_S48.split_loci_lrg_14
04132016_mw_1571_SC_B4_S48.split_loci_lrg_15
04132016_mw_1571_SC_B4_S48.split_loci_lrg_16
04132016_mw_1571_SC_B4_S48.split_loci_lrg_17
04132016_mw_1571_SC_B4_S48.split_loci_lrg_18
04132016_mw_1571_SC_B4_S48.split_loci_lrg_19

04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_45
04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_46
04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_47
04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_48
04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_49
50
04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_50
04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_51
04132016_mw_L1B1_SC_C1_S45.split_loci_lrg_52
/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/final/04132016_mw_L1B1_SC_C2_S46
39
0
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_00
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_01
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_02
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_03
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_04
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_05
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_06
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_07
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_08
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_09
10
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg_10
04132016_mw_L1B1_SC_C2_S46.split_loci_lrg

04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_73
04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_74
04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_75
04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_76
04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_77
04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_78
04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_79
80
04132016_mw_L1B1_SC_E2_S51.split_loci_lrg_80
/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/final/04132016_mw_L1B1_SC_E3_S52
66
0
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_00
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_01
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_02
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_03
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_04
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_05
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_06
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_07
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_08
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_09
10
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg_10
04132016_mw_L1B1_SC_E3_S52.split_loci_lrg

31
0
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_00
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_01
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_02
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_03
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_04
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_05
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_06
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_07
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_08
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_09
10
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_10
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_11
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_12
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_13
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_14
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_15
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_16
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_17
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_18
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_19
20
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_20
04132016_mw_L1B1_SC_H1_S55.split_loci_lrg_21

In [3]:
for cell in SingleCell_MALBAC:
    print cell
    os.chdir(os.path.join(basepath, diry, cell))
    for file in glob.glob("*s*__*.png"):
        newfile = re.sub("_s\d+__", "-", file)
        shutil.move(file, newfile)     
    for file in glob.glob("*.png"):
        img = Image.open(file)
        width = img.size[0]
        height = img.size[1]
        img2 = img.crop((70,130,width,height)).resize((200,500))
        path = os.path.splitext(file)[0]
        basename = os.path.basename(path)
        outfile1 = basename + "_crp.png"
        img2.save(outfile1)
        #os.remove(file)
    
    mergedpeak_data = os.path.join(basepath, diry, cell, cell + peaks_correct_data)
    regions_sml = os.path.join(basepath, diry, cell, cell + peakregions_sml)
    regions_lrg = os.path.join(basepath, diry, cell, cell + peakregions_lrg)
      
    count=1
    with open(mergedpeak_data) as r0,open(regions_sml) as r_sml,open(regions_lrg) as r_lrg:
        Files= {}
        for rr0,rr_sml,rr_lrg in zip(r0,r_sml,r_lrg):
            line = rr0.strip().split('\t')[0]+"\t"+rr0.strip().split('\t')[1]+"\t"+rr0.strip().split('\t')[2]+"\t"+cell+"_sml-"+rr_sml.strip().split('\t')[0]+"_"+rr_sml.strip().split('\t')[1]+"_"+rr_sml.strip().split('\t')[2]+"_crp.png"+"\t"+cell+"_lrg-"+rr_lrg.strip().split('\t')[0]+"_"+rr_lrg.strip().split('\t')[1]+"_"+rr_lrg.strip().split('\t')[2]+"_crp.png"+"\t"+rr0.strip().split('\t')[3]
            Files[str(count)] = line
            count+=1
    
    a = os.path.join(basepath, diry, cell, cell+"_overlap_0_binary")
    b = os.path.join(basepath, diry, cell, cell+"_overlap_1_binary")
    c = os.path.join(basepath, diry, cell, cell+"_overlap_2_binary")
    d = os.path.join(basepath, diry, cell, cell+"_overlap_3_binary")
    e = os.path.join(basepath, diry, cell, cell+"_overlap_4_binary")
    f = os.path.join(basepath, diry, cell, cell+"_overlap_5_binary")
    count=1
    with open(a) as f1,open(b) as f2,open(c) as f3,open(d) as f4,open(e) as f5,open(f) as f6:
        Peaks = {}
        for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
            line = aa.strip().split('\t')[6]+bb.strip().split('\t')[6]+cc.strip().split('\t')[6]+"\t"+dd.strip().split('\t')[6]+ee.strip().split('\t')[6]+ff.strip().split('\t')[6]
            Peaks[str(count)] = line
            count+=1
    
    a_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_0_binary")
    b_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_1_binary")
    c_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_2_binary")
    d_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_3_binary")
    e_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_4_binary")
    f_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_5_binary")
    count=1
    with open(a_sml) as f1,open(b_sml) as f2,open(c_sml) as f3,open(d_sml) as f4,open(e_sml) as f5,open(f_sml) as f6:
        Small = {}
        for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
            line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
            Small[str(count)] = line
            count+=1
    
    a_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_0_binary")
    b_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_1_binary")
    c_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_2_binary")
    d_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_3_binary")
    e_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_4_binary")
    f_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_5_binary")
    count=1
    with open(a_lrg) as f1,open(b_lrg) as f2,open(c_lrg) as f3,open(d_lrg) as f4,open(e_lrg) as f5,open(f_lrg) as f6:
        Large = {}
        for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
            line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
            Large[str(count)] = line
            count+=1
    
    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt"),"w") as f8:
        for key in Files:
            encoding = Files[key]+":"+Peaks[key].strip().split('\t')[0]+":"+Large[key].strip().split('\t')[0]+":"+Peaks[key].strip().split('\t')[1]+"\n"
        #for (Fi, Fj),(Pi,Pj),(Si,Sj),(Li,Lj) in zip(Files.items(),Peaks.items(),Small.items(),Large.items()):
            #encoding = Fj+":"+Pj.strip().split('\t')[0]+":"+Sj.strip().split('\t')[0]+":"+Lj.strip().split('\t')[0]+":"+Pj.strip().split('\t')[1]+":"+Sj.strip().split('\t')[1]+":"+Lj.strip().split('\t')[1]+"\n"
            f8.write(encoding)
            
#    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
#        for line in csv.reader(f, delimiter="\t"):
#            if os.path.isfile(line[3]):
#                filename = line[3]
#                dst = line[5]
#                if not os.path.exists(dst):
#                    os.makedirs(os.path.join(basepath, diry, cell, dst))
#                shutil.move(filename, os.path.join(basepath, diry, cell, dst))
                       
#    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
#        for line in csv.reader(f, delimiter="\t"):
#            if os.path.isfile(line[4]):
#                filename = line[4]
#                dst = line[5]
#                if not os.path.exists(dst):
#                    os.makedirs(os.path.join(basepath, diry, cell, dst))
#                shutil.move(filename, os.path.join(basepath, diry, cell, dst))                
        

04132016_mw_1571_SC_B4_S48
04132016_mw_1571_SC_D4_S49
04132016_mw_L1B1_SC_A2_S43
04132016_mw_L1B1_SC_C1_S45
04132016_mw_L1B1_SC_C2_S46
04132016_mw_L1B1_SC_D2_S50
04132016_mw_L1B1_SC_E2_S51
04132016_mw_L1B1_SC_E3_S52
04132016_mw_L1B1_SC_F2_S53
04132016_mw_L1B1_SC_G1_S54
04132016_mw_L1B1_SC_H1_S55
05252016_mw_L1B1_SC_B4_S47


In [10]:
print file

2265_S16_sml-Y_28245889_28250889.png


In [10]:
filelist_sml = []
metadata_sml = []
#metadata2_sml = []

for cell in SingleCell_MALBAC:
    print cell
    #shutil.copy(os.path.join(basepath, "final", cell, cell+"_Input_metadata.txt"),os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt"))
    os.chdir(os.path.join(basepath, diry, cell))
## Load images and meta data from meta files
    
    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
        for line in csv.reader(f, delimiter="\t"):
            if os.path.isfile(line[3]):
                readclass = line[5].split(":")[0]
                peakclass_sml = line[5].split(":")[1]               
                L1_class = line[5].split(":")[3]
                #print os.path.join(basepath, diry, cell, line[3])
                #if peakclass_sml == readclass:
                class_sml = [readclass,L1_class]
                filelist_sml.append(os.path.join(basepath, diry, cell, line[3])) 
                metadata_sml.append("".join(class_sml))
                #metadata2_sml.append("".join(L1_class))
                
os.chdir(os.path.join(basepath))
X_sml = np.array([np.array(Image.open(fname)) for fname in filelist_sml])
Y_sml = np.array(metadata_sml).astype(str)
print X_sml.shape
print Y_sml.size
print len(np.unique(Y_sml))
n_classes_sml = Y_sml.size
n_samples_sml = len(X_sml)
np.savez("MALBAC_Cells_sml", X_sml=X_sml, Y_sml=Y_sml)

#X2 = X.reshape((n_sam,-1))
#print X2.shape
#X = None



04132016_mw_1571_SC_B4_S48
04132016_mw_1571_SC_D4_S49
04132016_mw_L1B1_SC_A2_S43
04132016_mw_L1B1_SC_C1_S45
04132016_mw_L1B1_SC_C2_S46
04132016_mw_L1B1_SC_D2_S50
04132016_mw_L1B1_SC_E2_S51
04132016_mw_L1B1_SC_E3_S52
04132016_mw_L1B1_SC_F2_S53
04132016_mw_L1B1_SC_G1_S54
04132016_mw_L1B1_SC_H1_S55
05252016_mw_L1B1_SC_B4_S47
(30205, 500, 200, 3)
30205
31


In [11]:
print Y_sml

['011011' '011001' '011011' ..., '110000' '111010' '111010']


In [18]:
filelist_lrg = []
metadata_lrg = []

for cell in SingleCell_MALBAC:
    print cell
    #shutil.copy(os.path.join(basepath, "final", cell, cell+"_Input_metadata.txt"),os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt"))
    os.chdir(os.path.join(basepath, diry, cell))
## Load images and meta data from meta files
    
    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
        for line in csv.reader(f, delimiter="\t"):
            if os.path.isfile(line[4]):
                peakclass_lrg = line[5].split(":")[2]               
                L1_class = line[5].split(":")[3]
                #print os.path.join(basepath, diry, cell, line[3])
                filelist_lrg.append(os.path.join(basepath, diry, cell, line[4]))
                class_lrg = [peakclass_lrg,L1_class]
                metadata_lrg.append("".join(Class_lrg))
                
os.chdir(os.path.join(basepath))
X_lrg = np.array([np.array(Image.open(fname)) for fname in filelist_lrg])
Y_lrg = np.array(metadata_lrg).astype(str)
print X_lrg.shape
print Y_lrg.size
print len(np.unique(Y_lrg))
n_classes_lrg = Y_lrg.size
n_samples_lrg = len(X_lrg)
np.savez("MALBAC_Cells_lrg", X_lrg=X_lrg, Y_lrg=Y_lrg)

#X2 = X.reshape((n_sam,-1))
#print X2.shape
#X = None
#np.savez("All_Cells_sml", X2=X2, Y=Y)



04132016_mw_1571_SC_B4_S48
['211' '000' '101' '000' '111' '001' '101' '101' '211' '000' '001' '001'
 '000' '100' '200' '111' '211' '101' '101' '000' '100' '000' '001' '001'
 '212' '100' '101' '000' '211' '211' '001' '101' '100' '101' '002' '000'
 '210' '101' '101' '000' '000' '101' '111' '111' '001' '000' '001' '101'
 '111' '101' '001' '000' '100' '000' '101' '101' '000' '111' '101' '222'
 '221' '101' '111' '102' '100' '110' '100' '111' '112' '012' '112' '000'
 '100' '122' '001' '212' '110' '212' '100' '212' '100' '011' '000' '111'
 '111' '000' '111' '111' '011' '101' '000' '111' '111' '101' '111' '111'
 '111' '211' '000' '111' '111' '211' '000' '211' '001' '000' '101' '000'
 '000' '111' '111' '101' '000' '111' '000' '201' '100' '101' '101' '101'
 '001' '100' '101' '001' '101' '100' '222' '101' '111' '212' '111' '111'
 '011' '111' '111' '011' '001' '212' '101' '101' '100' '100' '101' '100'
 '101' '001' '111' '000' '100' '011' '100' '100' '211' '200' '100' '112'
 '211' '101' '011' '000'

In [ ]:
####Test Basic Convolution Model

import random
np.random.seed(123)  # for reproducibility
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

config = tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32, allow_soft_placement=True, device_count = {'CPU': 32})
session = tf.Session(config=config)
K.set_session(session)

# 6. Preprocess class labels
Ydict_1 = {}
Ydict_2 = {}
Y_unique = np.unique(Y_sml)
print Y_unique
i=0
for y_u in Y_unique:
    Ydict_1[i] = y_u
    Ydict_2[y_u] = i
    i+=1
print Ydict_2
trainsize = 20000
testsize = 10000
train_test = random.sample(range(0, n_samples_sml),trainsize+testsize)
train = train_test[0:trainsize]
test = train_test[trainsize:trainsize+testsize]
X_train = []
X_test = []
X_train = X_sml[train,:]
X_test = X_sml[test,:]
Ytr = [Ydict_2[y_train] for y_train in Y_sml[train]]
Yte = [Ydict_2[y_test] for y_test in Y_sml[test]]
Y_train = np_utils.to_categorical(Ytr)
Y_test = np_utils.to_categorical(Yte)

num_categories = len(np.unique(Y_sml))
print num_categories
# 7. Define model architecture
model = Sequential() 
model.add(Conv2D(32, (3,3), activation='relu', data_format="channels_first", input_shape=(3,500,200)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

# 8. Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

X_train = X_train.reshape(X_train.shape[0], 3, 500, 200)
X_test = X_test.reshape(X_test.shape[0], 3, 500, 200)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# 9. Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

score = model.evaluate(X_test, Y_test, verbose=1)

score

classes = model.predict(X_test, batch_size=128)

classes

X_test.shape

classes.shape

classes[1,]

['010000' '010001' '010010' '010011' '011000' '011001' '011010' '011011'
 '011100' '100000' '100001' '100010' '100011' '100100' '101000' '101001'
 '101010' '101011' '101101' '110000' '110001' '110010' '110011' '111000'
 '111001' '111010' '111011' '111100' '111101' '111110' '111111']
{'110000': 19, '110001': 20, '011011': 7, '011010': 6, '100100': 13, '111100': 27, '111101': 28, '100010': 11, '100011': 12, '111001': 24, '111000': 23, '101011': 17, '101010': 16, '110011': 22, '110010': 21, '010010': 2, '010011': 3, '011000': 4, '011001': 5, '011100': 8, '101101': 18, '111111': 30, '111110': 29, '100001': 10, '100000': 9, '010001': 1, '010000': 0, '111010': 25, '111011': 26, '101000': 14, '101001': 15}
31
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 498, 198)      896       
_________________________________________________________________
conv2d_5 (Conv2D)       

In [ ]:
####Test Basic Convolution Model

import random
np.random.seed(123)  # for reproducibility
import tensorflow as tf
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

config = tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32, allow_soft_placement=True, device_count = {'CPU': 32})
session = tf.Session(config=config)
K.set_session(session)

# 6. Preprocess class labels
Ydict_1 = {}
Ydict_2 = {}
Y_unique = np.unique(Y1_1)
print Y_unique
i=0
for y_u in Y_unique:
    Ydict_1[i] = y_u
    Ydict_2[y_u] = i
    i+=1
print Ydict_2
trainsize = 20000
testsize = 10000
train_test = random.sample(range(0, n_sam),trainsize+testsize)
train = train_test[0:trainsize]
test = train_test[trainsize:trainsize+testsize]
X_train = []
X_test = []
X_train = X1[train,:]
X_test = X1[test,:]
Ytr = [Ydict_2[y_train] for y_train in Y1_1[train]]
Yte = [Ydict_2[y_test] for y_test in Y1_1[test]]
Y_train = np_utils.to_categorical(Ytr)
Y_test = np_utils.to_categorical(Yte)

num_categories = len(np.unique(Y1_1))
print num_categories
# 7. Define model architecture
model = InceptionV3(weights='None', include_top=False)

# 8. Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

X_train = X_train.reshape(X_train.shape[0], 3, 500, 200)
X_test = X_test.reshape(X_test.shape[0], 3, 500, 200)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# 9. Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1)

score = model.evaluate(X_test, Y_test, verbose=1)

score

classes = model.predict(X_test, batch_size=128)

classes

X_test.shape

classes.shape

classes[1,]

In [13]:
print score

[1.088644332408905, 0.56600000047683718]


In [12]:
print classes.shape

(500, 5)
